In [ ]:
# default_exp utils

# Utils

> Utility functions for Deep Animator library.

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
# export
import yaml
import torch
import torch.nn as nn

from typing import Tuple

from deep_animator.modules.generator import OcclusionAwareGenerator
from deep_animator.sync_batchnorm.replicate import DataParallelWithCallback

In [ ]:
# export
def load_checkpoints(config_path: str, checkpoint_path: str, device: str = 'cpu') \
                     -> Tuple[torch.nn.Module, torch.nn.Module]:
    # load configuration
    with open(config_path) as f:
        config = yaml.load(f, Loader=yaml.FullLoader)

    generator = OcclusionAwareGenerator(**config['model_params']['generator_params'],
                                        **config['model_params']['common_params'])
    generator.to(device)

    kp_detector = KPDetector(**config['model_params']['kp_detector_params'],
                             **config['model_params']['common_params'])
    kp_detector.to(device)

    checkpoint = torch.load(checkpoint_path)
    generator.load_state_dict(checkpoint['generator'])
    kp_detector.load_state_dict(checkpoint['kp_detector'])

    generator = DataParallelWithCallback(generator)
    kp_detector = DataParallelWithCallback(kp_detector)

    generator.eval()
    kp_detector.eval()
    
    return generator, kp_detector